## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-24-16-53-35 +0000,nypost,Business Insider will start publishing stories...,https://nypost.com/2025/10/24/business/busines...
1,2025-10-24-16-52-42 +0000,bbc,Plaid wins by-election as Labour loses 100-yea...,https://www.bbc.com/news/articles/cd9klevy28qo...
2,2025-10-24-16-52-00 +0000,wsj,The Pentagon said it received $130 million fro...,https://www.wsj.com/politics/policy/anonymous-...
3,2025-10-24-16-49-28 +0000,nypost,Dem Jersey City mayor wants Mike Sherrill to r...,https://nypost.com/2025/10/24/us-news/dem-jers...
4,2025-10-24-16-48-00 +0000,wsj,Pentagon Orders Aircraft Carrier to the Caribbean,https://www.wsj.com/world/americas/pentagon-or...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2360/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
37,trump,62
2,will,21
188,new,18
299,off,16
92,canada,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
220,2025-10-24-03-21-49 +0000,cbc,Trump claims Canada 'CHEATED AND GOT CAUGHT' w...,https://www.cbc.ca/news/world/trump-trade-nego...,148
264,2025-10-23-23-29-06 +0000,nypost,Ford warns devasting plant fire will hurt prof...,https://nypost.com/2025/10/23/business/ford-wa...,132
221,2025-10-24-03-16-11 +0000,nypost,Trump terminates trade talks with Canada over ...,https://nypost.com/2025/10/23/us-news/trump-te...,132
121,2025-10-24-12-13-55 +0000,cbc,LIVE UPDATES | Carney says Canada prepared to ...,https://www.cbc.ca/news/canada/livestory/live-...,126
95,2025-10-24-13-31-52 +0000,wapo,Trump says he’s canceling trade negotiations w...,https://www.washingtonpost.com/world/2025/10/2...,125


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
220,148,2025-10-24-03-21-49 +0000,cbc,Trump claims Canada 'CHEATED AND GOT CAUGHT' w...,https://www.cbc.ca/news/world/trump-trade-nego...
264,70,2025-10-23-23-29-06 +0000,nypost,Ford warns devasting plant fire will hurt prof...,https://nypost.com/2025/10/23/business/ford-wa...
50,69,2025-10-24-15-22-00 +0000,wsj,Annual inflation heated up slightly in Septemb...,https://www.wsj.com/economy/consumer-price-ind...
118,54,2025-10-24-12-24-07 +0000,nypost,Wild video shows security shoot at U-Haul truc...,https://nypost.com/2025/10/24/us-news/video-sh...
327,52,2025-10-23-20-01-18 +0000,nypost,The corporations and donors footing the bill f...,https://nypost.com/2025/10/23/us-news/here-are...
60,51,2025-10-24-14-59-00 +0000,wsj,The extended government shutdown has put colle...,https://www.wsj.com/economy/shutdown-puts-octo...
343,50,2025-10-23-18-09-53 +0000,nypost,"China, India curb Russian oil purchases after ...",https://nypost.com/2025/10/23/world-news/china...
90,41,2025-10-24-13-44-56 +0000,nypost,Six dead after US strike in Caribbean targets ...,https://nypost.com/2025/10/24/us-news/6-dead-a...
111,41,2025-10-24-13-00-06 +0000,wapo,Rubio stresses there is ‘no plan B’ amid quest...,https://www.washingtonpost.com/world/2025/10/2...
53,39,2025-10-24-15-15-04 +0000,nypost,NY Attorney General Letitia James pleads not g...,https://nypost.com/2025/10/24/us-news/new-york...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
